# FVC Deepfake Detection: Complete Production Pipeline

**From Raw ZIP Archives to Production-Ready ML Models**

This comprehensive notebook demonstrates a production-grade machine learning pipeline for deepfake video detection, showcasing:
- **Data Engineering**: Extraction, validation, and preprocessing
- **Feature Engineering**: Handcrafted features with domain expertise
- **Model Architecture**: 23 diverse models from baselines to state-of-the-art
- **MLOps Infrastructure**: MLflow, Airflow, DuckDB integration
- **Production Practices**: 5-fold CV, hyperparameter optimization, experiment tracking

**Target Audience**: ML Engineers, Research Scientists, Hiring Managers
**Level**: Production-Grade, Research-Quality Implementation

## Table of Contents

1. [Infrastructure & Requirements](#1-infrastructure--requirements)
2. [Data Extraction & Exploration](#2-data-extraction--exploration)
3. [Stage 1: Video Augmentation Strategy](#3-stage-1-video-augmentation-strategy)
4. [Stage 2: Handcrafted Feature Engineering](#4-stage-2-handcrafted-feature-engineering)
5. [Stage 3: Video Scaling & Normalization](#5-stage-3-video-scaling--normalization)
6. [Stage 4: Scaled Feature Extraction](#6-stage-4-scaled-feature-extraction)
7. [Stage 5: Model Training Architecture](#7-stage-5-model-training-architecture)
8. [MLOps: Experiment Tracking with MLflow](#8-mlops-experiment-tracking-with-mlflow)
9. [Analytics: DuckDB for Fast Queries](#9-analytics-duckdb-for-fast-queries)
10. [Orchestration: Apache Airflow DAGs](#10-orchestration-apache-airflow-dags)
11. [Model Evaluation & Results](#11-model-evaluation--results)
12. [Production Deployment Considerations](#12-production-deployment-considerations)

## 1. Infrastructure & Requirements

### Technology Stack

**Deep Learning Framework**:
- PyTorch 2.0+ with CUDA 11.8+ support
- torchvision 0.15+ for video models (X3D, SlowFast, R(2+1)D, I3D)
- timm 0.9+ for Vision Transformers (ViT, TimeSformer, ViViT)
- transformers 4.30+ for HuggingFace model integration

**Data Processing Stack**:
- **Polars 0.19+**: Columnar DataFrame library (10-100x faster than pandas)
- **PyArrow 14+**: In-memory columnar format (Arrow) and file format (Parquet)
- **DuckDB 0.9+**: In-process analytical SQL database for fast queries
- **Pandera 0.18+**: DataFrame schema validation

**MLOps & Orchestration**:
- **MLflow 2.8+**: Experiment tracking, model registry, artifact management
- **Apache Airflow 2.7+**: Workflow orchestration, dependency management, scheduling
- **Custom MLOps**: ExperimentTracker, CheckpointManager, RunConfig

**Video Processing**:
- **PyAV 10.0+**: Pythonic FFmpeg bindings for efficient video I/O
- **OpenCV 4.8+**: Computer vision operations (feature extraction, transforms)
- **FFmpeg/ffprobe**: Codec analysis, metadata extraction
- **PyTorchVideo 0.1.5+**: Video model library (I3D, X3D, SlowFast)

**Feature Engineering**:
- **NumPy 1.24+**: Signal processing, DCT transforms
- **scikit-image**: Image analysis, filters
- **scipy 1.11+**: Statistical functions

**Machine Learning**:
- **scikit-learn 1.3+**: Baseline models (Logistic Regression, SVM)
- **XGBoost 2.0+**: Gradient boosting with pretrained feature extractors
- **joblib 1.3+**: Model serialization

In [ ]:
import sys
from pathlib import Path
import json
import numpy as np
import pandas as pd
import polars as pl
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import display, HTML, Video, Image
import warnings
warnings.filterwarnings('ignore')

# Set style
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

# Add project root to path
project_root = Path().absolute().parent.parent
sys.path.insert(0, str(project_root))

print(f"📁 Project root: {project_root}")
print(f"🐍 Python version: {sys.version.split()[0]}")
print(f"\n✅ All imports successful")

In [ ]:
# Verify infrastructure stack
import torch
import torchvision
import polars as pl
import pyarrow as pa

infrastructure_status = {}

# Deep Learning
infrastructure_status['PyTorch'] = f"{torch.__version__}"
infrastructure_status['torchvision'] = f"{torchvision.__version__}"
if torch.cuda.is_available():
    infrastructure_status['CUDA'] = f"{torch.version.cuda} ({torch.cuda.get_device_name(0)})"
else:
    infrastructure_status['CUDA'] = "Not available (CPU mode)"

# Data Processing
infrastructure_status['Polars'] = pl.__version__
infrastructure_status['PyArrow'] = pa.__version__

# MLOps
try:
    import mlflow
    infrastructure_status['MLflow'] = mlflow.__version__
except ImportError:
    infrastructure_status['MLflow'] = "Not installed"

try:
    import duckdb
    infrastructure_status['DuckDB'] = duckdb.__version__
except ImportError:
    infrastructure_status['DuckDB'] = "Not installed"

# Display status
status_df = pd.DataFrame(list(infrastructure_status.items()), columns=['Component', 'Version'])
display(status_df.style.set_properties(**{'text-align': 'left'}).set_table_styles([
    {'selector': 'th', 'props': [('background-color', '#4CAF50'), ('color', 'white'), ('font-weight', 'bold')]}
]))

## 2. Data Extraction & Exploration

### Initial Data Structure

The FVC (Fake Video Challenge) dataset comes as password-protected ZIP archives:
- `FVC1.zip`, `FVC2.zip`, `FVC3.zip`: Video files (MP4 format)
- `Metadata.zip`: CSV metadata files with labels (real/fake)

### Data Extraction Process

**Location**: `src/setup_fvc_dataset.py`

**Steps**:
1. Extract videos from ZIP archives to `data/videos/`
2. Copy metadata CSV files to `archive/`
3. Build comprehensive video index with:
   - File paths and sizes
   - Video metadata (duration, fps, resolution, codec)
   - Labels (real/fake)
   - Data integrity checks
4. Generate metadata: `data/videos/video_index.parquet`

### Data Exploration Rationale

**Why Explore Before Processing?**
- **Class Distribution**: Check for imbalance (affects loss functions, sampling)
- **Video Statistics**: Duration, resolution, codec diversity (affects preprocessing)
- **Data Quality**: Corrupted files, missing metadata (affects pipeline robustness)
- **Storage Requirements**: Estimate disk space for augmented/scaled videos
- **Processing Time**: Estimate pipeline duration based on video counts

In [ ]:
# Check for archive files and extracted data
archive_dir = project_root / "archive"
data_dir = project_root / "data"
videos_dir = data_dir / "videos"

print("📦 Archive Directory:")
if archive_dir.exists():
    zip_files = list(archive_dir.glob("*.zip"))
    csv_files = list(archive_dir.glob("*.csv"))
    
    print(f"  ✓ Found {len(zip_files)} ZIP archives")
    for f in zip_files:
        size_gb = f.stat().st_size / (1024**3)
        print(f"    - {f.name}: {size_gb:.2f} GB")
    
    print(f"\n  ✓ Found {len(csv_files)} CSV metadata files")
    for f in csv_files:
        print(f"    - {f.name}")
else:
    print("  ⚠ Archive directory not found")

print("\n📁 Data Directory:")
if videos_dir.exists():
    video_files = list(videos_dir.glob("*.mp4"))
    index_file = videos_dir / "video_index.parquet"
    
    print(f"  ✓ Found {len(video_files)} video files")
    print(f"  ✓ Index file exists: {index_file.exists()}")
    
    if index_file.exists():
        from lib.utils.paths import load_metadata_flexible
        index_df = load_metadata_flexible(str(index_file))
        if index_df is not None:
            print(f"\n  📊 Video Index Statistics:")
            print(f"    - Total videos: {index_df.height}")
            if 'label' in index_df.columns:
                label_counts = index_df['label'].value_counts()
                print(f"    - Class distribution:")
                for label, count in label_counts.items():
                    print(f"      {label}: {count} ({100*count/index_df.height:.1f}%)")
else:
    print("  ⚠ Videos directory not found - run setup_fvc_dataset.py first")

In [ ]:
# Load and explore metadata
if videos_dir.exists() and (videos_dir / "video_index.parquet").exists():
    from lib.utils.paths import load_metadata_flexible
    
    index_df = load_metadata_flexible(str(videos_dir / "video_index.parquet"))
    
    if index_df is not None and index_df.height > 0:
        # Convert to pandas for visualization
        index_pd = index_df.to_pandas()
        
        # Class distribution
        if 'label' in index_pd.columns:
            fig, axes = plt.subplots(1, 2, figsize=(12, 4))
            
            # Bar plot
            label_counts = index_pd['label'].value_counts()
            axes[0].bar(label_counts.index, label_counts.values, color=['#4CAF50', '#f44336'])
            axes[0].set_title('Class Distribution', fontsize=14, fontweight='bold')
            axes[0].set_xlabel('Label')
            axes[0].set_ylabel('Count')
            axes[0].grid(axis='y', alpha=0.3)
            
            # Pie chart
            axes[1].pie(label_counts.values, labels=label_counts.index, autopct='%1.1f%%', 
                       colors=['#4CAF50', '#f44336'], startangle=90)
            axes[1].set_title('Class Proportion', fontsize=14, fontweight='bold')
            
            plt.tight_layout()
            plt.show()
            
            # Video statistics
            if 'duration' in index_pd.columns:
                fig, axes = plt.subplots(1, 2, figsize=(12, 4))
                
                # Duration distribution
                axes[0].hist(index_pd['duration'], bins=30, color='#2196F3', edgecolor='black', alpha=0.7)
                axes[0].set_title('Video Duration Distribution', fontsize=14, fontweight='bold')
                axes[0].set_xlabel('Duration (seconds)')
                axes[0].set_ylabel('Frequency')
                axes[0].grid(axis='y', alpha=0.3)
                
                # FPS distribution
                if 'fps' in index_pd.columns:
                    axes[1].hist(index_pd['fps'], bins=30, color='#FF9800', edgecolor='black', alpha=0.7)
                    axes[1].set_title('Frame Rate Distribution', fontsize=14, fontweight='bold')
                    axes[1].set_xlabel('FPS')
                    axes[1].set_ylabel('Frequency')
                    axes[1].grid(axis='y', alpha=0.3)
                
                plt.tight_layout()
                plt.show()
        
        print("\n📈 Summary Statistics:")
        display(index_pd.describe())

## 3. Stage 1: Video Augmentation Strategy

### Why Augmentation?

**Problem**: Limited dataset size (typically 200-500 videos)
- **Overfitting Risk**: Small datasets lead to poor generalization
- **Limited Diversity**: Real-world videos have infinite variations
- **Class Imbalance**: May need more samples of minority class

**Solution**: Data augmentation to increase dataset diversity
- **10x Dataset Expansion**: Generate 10 augmented versions per video
- **Spatial Diversity**: Rotation, flip, color jitter, noise, blur
- **Temporal Diversity**: Frame dropping, duplication, reversal

### Augmentation Types & Rationale

**Spatial Augmentations** (applied per-frame):

1. **Rotation (±10°)**: Simulates camera angle variation, handles tilted videos
2. **Horizontal Flip**: Doubles dataset, preserves temporal structure (no semantic change for faces)
3. **Brightness/Contrast/Saturation Jitter**: Handles lighting variations, different cameras
4. **Gaussian Noise**: Adds robustness to compression artifacts, low-quality captures
5. **Gaussian Blur**: Simulates motion blur, out-of-focus captures
6. **Affine Transformations**: Translation, scale, shear (handles camera movement)
7. **Elastic Transform**: Simulates non-rigid deformations (handles perspective changes)
8. **Cutout (Random Erasing)**: Occlusion robustness, prevents overfitting to specific regions

**Temporal Augmentations** (applied to sequence):

1. **Frame Dropping (up to 25%)**: Handles variable frame rates, temporal compression
2. **Frame Duplication**: Slow motion effect, temporal interpolation
3. **Temporal Reversal**: Time-reversed videos (doubles temporal diversity)

### Implementation: Pre-Generated vs On-the-Fly

**Why Pre-Generated Augmentations?**

**Advantages**:
- ✅ **Reproducibility**: Same augmentations across runs (deterministic seeds)
- ✅ **Speed**: No augmentation overhead during training (10-100x faster)
- ✅ **Caching**: Can store on disk, share across experiments
- ✅ **Memory Efficiency**: Frame-by-frame decoding (50x memory reduction vs loading full videos)
- ✅ **Debugging**: Can inspect augmented videos before training

**Trade-offs**:
- ⚠️ **Disk Space**: 10x dataset size (mitigated by scaling to 256px max dimension)
- ⚠️ **Initial Processing Time**: One-time cost (parallelizable)

**Location**: `lib/augmentation/pipeline.py`, `lib/augmentation/transforms.py`

In [ ]:
# Check for augmented videos
augmented_dir = data_dir / "augmented_videos"
augmented_metadata = augmented_dir / "augmented_metadata.parquet"

if augmented_metadata.exists():
    from lib.utils.paths import load_metadata_flexible
    
    aug_df = load_metadata_flexible(str(augmented_metadata))
    
    if aug_df is not None:
        print(f"✅ Stage 1 Complete: {aug_df.height} augmented videos")
        
        # Show augmentation statistics
        if 'augmentation_type' in aug_df.columns:
            aug_counts = aug_df['augmentation_type'].value_counts()
            print(f"\n📊 Augmentation Type Distribution:")
            for aug_type, count in aug_counts.items():
                print(f"  - {aug_type}: {count}")
        
        # Sample augmented video
        sample_video = aug_df.filter(pl.col('label') == 'real').head(1)
        if sample_video.height > 0:
            video_path = sample_video['video_path'][0]
            print(f"\n🎬 Sample Augmented Video: {Path(video_path).name}")
            # Note: Video display requires actual video file
            # Video(video_path, width=400)
else:
    print("⚠️ Stage 1 not completed - augmented videos not found")
    print("\n💡 To run Stage 1:")
    print("```python")
    print("from lib.augmentation.pipeline import stage1_augment_videos")
    print("")
    print("stage1_augment_videos(")
    print("    project_root='.',")
    print("    num_augmentations=10,")
    print("    output_dir='data/augmented_videos'")
    print(")")
    print("```")

## 4. Stage 2: Handcrafted Feature Engineering

### Why Handcrafted Features?

**Domain Knowledge**: Deepfake videos exhibit specific artifacts that can be detected:
- **Compression Artifacts**: Block boundaries, DCT patterns
- **Face Swap Artifacts**: Inconsistencies at boundaries
- **Temporal Inconsistencies**: Frame-to-frame variations
- **Codec Cues**: Compression parameters differ between real and fake

**Advantages**:
- **Interpretability**: Features have clear meaning
- **Efficiency**: Fast extraction, small feature vectors (~50 features)
- **Baseline Models**: Enable simple models (Logistic Regression, SVM)
- **Complementary**: Can be combined with deep learning features

### Feature Types & Extraction Methods

**1. Noise Residual Energy** (3 features)
- **Method**: High-pass filter to extract noise patterns
- **Rationale**: Deepfakes often have different noise characteristics
- **Features**: Total energy, mean energy, std energy

**2. DCT Band Statistics** (5 features)
- **Method**: Discrete Cosine Transform on 8x8 blocks (JPEG-like)
- **Rationale**: Compression artifacts differ between real and fake
- **Features**: DC coefficient mean/std, AC coefficient mean/std/energy

**3. Blur/Sharpness Metrics** (3 features)
- **Method**: Laplacian variance (sharpness), gradient mean (edge strength)
- **Rationale**: Deepfakes may have different sharpness characteristics
- **Features**: Laplacian variance, gradient mean, gradient std

**4. Block Boundary Inconsistency** (1 feature)
- **Method**: Detect inconsistencies at 8x8 block boundaries
- **Rationale**: Face swap boundaries create artifacts
- **Features**: Boundary inconsistency score

**5. Codec Cues** (3 features)
- **Method**: FFprobe analysis of video codec parameters
- **Rationale**: Real and fake videos may use different codecs/parameters
- **Features**: Codec type, bitrate, GOP size

**Total**: ~15 features per video (aggregated across frames)

**Location**: `lib/features/handcrafted.py`, `lib/features/pipeline.py`

In [ ]:
# Check for Stage 2 features
features_dir = data_dir / "features_stage2"
features_metadata = features_dir / "features_metadata.parquet"

if features_metadata.exists():
    from lib.utils.paths import load_metadata_flexible
    
    features_df = load_metadata_flexible(str(features_metadata))
    
    if features_df is not None:
        print(f"✅ Stage 2 Complete: {features_df.height} feature vectors")
        
        # Load a sample feature vector
        sample_row = features_df.head(1)
        if 'feature_path' in sample_row.columns:
            feature_path = sample_row['feature_path'][0]
            try:
                features = np.load(feature_path)
                if isinstance(features, dict):
                    print(f"\n📊 Sample Feature Vector ({len(features)} features):")
                    for key, value in list(features.items())[:10]:
                        if isinstance(value, (int, float)):
                            print(f"  - {key}: {value:.6f}")
                        else:
                            print(f"  - {key}: {type(value).__name__}")
                else:
                    print(f"\n📊 Feature vector shape: {features.shape}")
            except Exception as e:
                print(f"⚠️ Could not load features: {e}")
else:
    print("⚠️ Stage 2 not completed - features not found")
    print("\n💡 To run Stage 2:")
    print("```python")
    print("from lib.features.pipeline import stage2_extract_features")
    print("")
    print("stage2_extract_features(")
    print("    project_root='.',")
    print("    augmented_metadata_path='data/augmented_videos/augmented_metadata.parquet',")
    print("    output_dir='data/features_stage2'")
    print(")")
    print("```")

## 5. Stage 3: Video Scaling & Normalization

### Why Scale Videos?

**Problem**: Videos have diverse resolutions (e.g., 1920x1080, 640x480, 1280x720)
- **Memory Constraints**: Full-resolution videos require 10-100GB GPU memory
- **Model Input Requirements**: Most models expect fixed-size inputs (e.g., 256x256)
- **Training Speed**: Smaller videos train 10-100x faster

**Solution**: Scale all videos to target max dimension while preserving aspect ratio

### Scaling Strategy

**Target Resolution**: 256x256 (max dimension = 256px)
- **Downscaling**: Large videos (e.g., 1920x1080 → 256x144) reduce memory
- **Upscaling**: Small videos (e.g., 320x240 → 256x192) ensure minimum quality
- **Aspect Ratio Preservation**: Letterboxing maintains original proportions

### Scaling Methods

**1. Letterbox Resize** (Default)
- **Method**: Bilinear interpolation with letterboxing (black bars)
- **Pros**: Fast, simple, preserves aspect ratio
- **Cons**: Black bars waste pixels
- **Use Case**: Production default, fastest option

**2. Autoencoder Upscaling** (Optional)
- **Method**: Pretrained HuggingFace VAE for high-quality upscaling
- **Pros**: Better quality for upscaled videos
- **Cons**: Slower, requires GPU
- **Use Case**: Research, quality-critical applications

### Normalization

**Pixel Normalization**:
- **Method**: Normalize to [0, 1] or ImageNet statistics
- **Rationale**: Consistent input distribution improves training stability
- **Implementation**: Applied during DataLoader transforms

**Location**: `lib/scaling/pipeline.py`, `lib/scaling/methods.py`

In [ ]:
# Check for scaled videos
scaled_dir = data_dir / "scaled_videos"
scaled_metadata = scaled_dir / "scaled_metadata.parquet"

if scaled_metadata.exists():
    from lib.utils.paths import load_metadata_flexible
    
    scaled_df = load_metadata_flexible(str(scaled_metadata))
    
    if scaled_df is not None:
        print(f"✅ Stage 3 Complete: {scaled_df.height} scaled videos")
        
        # Show resolution statistics
        if 'scaled_width' in scaled_df.columns and 'scaled_height' in scaled_df.columns:
            scaled_pd = scaled_df.select(['scaled_width', 'scaled_height']).to_pandas()
            
            fig, axes = plt.subplots(1, 2, figsize=(12, 4))
            
            # Width distribution
            axes[0].hist(scaled_pd['scaled_width'], bins=30, color='#2196F3', edgecolor='black', alpha=0.7)
            axes[0].set_title('Scaled Width Distribution', fontsize=14, fontweight='bold')
            axes[0].set_xlabel('Width (pixels)')
            axes[0].set_ylabel('Frequency')
            axes[0].grid(axis='y', alpha=0.3)
            axes[0].axvline(256, color='red', linestyle='--', label='Target: 256px')
            axes[0].legend()
            
            # Height distribution
            axes[1].hist(scaled_pd['scaled_height'], bins=30, color='#FF9800', edgecolor='black', alpha=0.7)
            axes[1].set_title('Scaled Height Distribution', fontsize=14, fontweight='bold')
            axes[1].set_xlabel('Height (pixels)')
            axes[1].set_ylabel('Frequency')
            axes[1].grid(axis='y', alpha=0.3)
            axes[1].axvline(256, color='red', linestyle='--', label='Target: 256px')
            axes[1].legend()
            
            plt.tight_layout()
            plt.show()
else:
    print("⚠️ Stage 3 not completed - scaled videos not found")
    print("\n💡 To run Stage 3:")
    print("```python")
    print("from lib.scaling.pipeline import stage3_scale_videos")
    print("")
    print("stage3_scale_videos(")
    print("    project_root='.',")
    print("    augmented_metadata_path='data/augmented_videos/augmented_metadata.parquet',")
    print("    output_dir='data/scaled_videos',")
    print("    target_size=256")
    print(")")
    print("```")

## 6. Stage 4: Scaled Feature Extraction

### Why Extract Features from Scaled Videos?

**Complementary Information**:
- **Scale-Invariant Features**: Some artifacts are visible at different scales
- **Normalized Statistics**: Features extracted from normalized resolutions
- **Model Input Alignment**: Features match the scale used by video models

**Same Feature Types as Stage 2**:
- Noise residual energy
- DCT statistics
- Blur/sharpness metrics
- Block boundary inconsistency
- Codec cues

**Total Features**: ~15 features from scaled videos + ~15 from original = ~30 total handcrafted features

**Location**: `lib/features/scaled.py`

In [ ]:
# Check for Stage 4 features
features4_dir = data_dir / "features_stage4"
features4_metadata = features4_dir / "features_metadata.parquet"

if features4_metadata.exists():
    from lib.utils.paths import load_metadata_flexible
    
    features4_df = load_metadata_flexible(str(features4_metadata))
    
    if features4_df is not None:
        print(f"✅ Stage 4 Complete: {features4_df.height} scaled feature vectors")
else:
    print("⚠️ Stage 4 not completed - scaled features not found")
    print("\n💡 To run Stage 4:")
    print("```python")
    print("from lib.features.scaled import stage4_extract_scaled_features")
    print("")
    print("stage4_extract_scaled_features(")
    print("    project_root='.',")
    print("    scaled_metadata_path='data/scaled_videos/scaled_metadata.parquet',")
    print("    output_dir='data/features_stage4'")
    print(")")
    print("```")

## 7. Stage 5: Model Training Architecture

### Model Portfolio (23 Models)

**Baseline Models** (Feature-Based):
- **5a**: Logistic Regression (handcrafted features)
- **5b**: SVM (handcrafted features)

**CNN Models** (Direct Video Processing):
- **5c**: Naive CNN (3D convolutions, 1000 frames)
- **5d**: Pretrained Inception (R3D-18 backbone + Inception head)
- **5e**: Variable AR CNN (handles variable aspect ratios)

**XGBoost + Pretrained Feature Extractors**:
- **5f**: XGBoost + Pretrained Inception features
- **5g**: XGBoost + I3D features
- **5h**: XGBoost + R(2+1)D features
- **5i**: XGBoost + ViT-GRU features
- **5j**: XGBoost + ViT-Transformer features

**Vision Transformer Models**:
- **5k**: ViT-GRU (ViT per frame + GRU temporal)
- **5l**: ViT-Transformer (ViT per frame + Transformer temporal)
- **5m**: TimeSformer (divided space-time attention)
- **5n**: ViViT (tubelet embedding)

**3D CNN Models**:
- **5o**: I3D (Inflated 3D ConvNet)
- **5p**: R(2+1)D (Factorized 3D convolutions)
- **5q**: X3D (Efficient video models)

**SlowFast Variants**:
- **5r**: SlowFast (dual pathway: slow + fast)
- **5s**: SlowFast with Attention
- **5t**: Multi-Scale SlowFast

**Two-Stream Models**:
- **5u**: Two-Stream (RGB + Optical Flow)

### Training Strategy

**5-Fold Stratified Cross-Validation**:
- **Stratification**: Ensures balanced class distribution in each fold
- **Group-Aware Splitting**: Prevents data leakage (same video ID in train/val)
- **Reproducibility**: Fixed random seeds (42)

**Hyperparameter Optimization**:
- **Grid Search**: Exhaustive search over hyperparameter space
- **Sample-Based**: Grid search on 10-20% sample for efficiency
- **Best Params**: Applied to full dataset training
- **Single Combination**: Models 5c-5u use single hyperparameter set (efficiency)

**Regularization Techniques**:
- **L2 Regularization**: Weight decay (1e-4 to 1e-3)
- **Dropout**: 0.3-0.5 for fully connected layers
- **Batch Normalization**: Stabilizes training, enables higher learning rates
- **Gradient Clipping**: Prevents exploding gradients (max_norm=1.0)

**Optimization**:
- **Optimizer**: Adam with learning rate 1e-4 to 5e-4
- **Scheduler**: Cosine annealing with warmup (2 epochs)
- **Mixed Precision**: AMP for 2x speedup, 50% memory reduction
- **Gradient Accumulation**: Effective batch size = batch_size × accumulation_steps

**Activation Functions**:
- **ReLU**: Standard for CNNs
- **GELU**: For Transformers (smoother gradients)
- **Sigmoid**: Final output (binary classification)

**Location**: `lib/training/pipeline.py`, `lib/training/trainer.py`

In [ ]:
# Check for trained models
stage5_dir = data_dir / "stage5"

if stage5_dir.exists():
    model_dirs = [d for d in stage5_dir.iterdir() if d.is_dir()]
    
    print(f"✅ Found {len(model_dirs)} trained models:")
    
    model_status = []
    for model_dir in sorted(model_dirs):
        model_name = model_dir.name
        
        # Check for model files
        model_files = list(model_dir.glob("**/*.pt")) + list(model_dir.glob("**/*.joblib"))
        metrics_files = list(model_dir.glob("**/metrics.json"))
        
        status = {
            'Model': model_name,
            'Checkpoints': len(model_files),
            'Metrics': 'Yes' if metrics_files else 'No'
        }
        model_status.append(status)
    
    status_df = pd.DataFrame(model_status)
    display(status_df.style.set_properties(**{'text-align': 'left'}).set_table_styles([
        {'selector': 'th', 'props': [('background-color', '#2196F3'), ('color', 'white'), ('font-weight', 'bold')]}
    ]))
else:
    print("⚠️ Stage 5 not completed - no trained models found")
    print("\n💡 To train models:")
    print("```python")
    print("from lib.training.pipeline import stage5_train_models")
    print("")
    print("results = stage5_train_models(")
    print("    project_root='.',")
    print("    scaled_metadata_path='data/scaled_videos/scaled_metadata.parquet',")
    print("    features_stage2_path='data/features_stage2/features_metadata.parquet',")
    print("    features_stage4_path='data/features_stage4/features_metadata.parquet',")
    print("    model_types=['logistic_regression', 'svm', 'i3d', 'x3d', 'slowfast'],")
    print("    n_splits=5,")
    print("    num_frames=1000,")
    print("    output_dir='data/stage5',")
    print("    use_tracking=True,")
    print("    use_mlflow=True")
    print(")")
    print("```")

## 8. MLOps: Experiment Tracking with MLflow

### MLflow Integration

**What MLflow Tracks**:
- **Hyperparameters**: Learning rate, batch size, weight decay, etc.
- **Metrics**: Train/val/test loss, accuracy, F1, precision, recall
- **Artifacts**: Model checkpoints, configs, plots, logs
- **Metadata**: Run ID, experiment name, timestamps, tags

**Benefits**:
- **Reproducibility**: Track exact hyperparameters for each run
- **Comparison**: Compare models across experiments
- **Model Registry**: Version and manage production models
- **UI**: Web interface for browsing experiments

**Location**: `lib/mlops/mlflow_tracker.py`

In [ ]:
# Connect to MLflow
try:
    import mlflow
    
    # Set tracking URI (default: local file store)
    mlflow.set_tracking_uri("file:./mlruns")
    
    # List experiments
    experiments = mlflow.search_experiments()
    
    print(f"✅ MLflow Connected: {len(experiments)} experiments found")
    
    if len(experiments) > 0:
        print("\n📊 Recent Experiments:")
        for exp in experiments[:5]:
            print(f"  - {exp.name} (ID: {exp.experiment_id})")
        
        # Get runs from first experiment
        exp = experiments[0]
        runs = mlflow.search_runs(experiment_ids=[exp.experiment_id], max_results=10)
        
        if len(runs) > 0:
            print(f"\n📈 Recent Runs (showing top 10):")
            display(runs[['run_id', 'status', 'start_time', 'metrics.test_f1', 'params.model_type']].head(10))
    
    print("\n💡 To start MLflow UI:")
    print("```bash")
    print("mlflow ui --port 5000")
    print("# Open http://localhost:5000")
    print("```")
    
except ImportError:
    print("⚠️ MLflow not installed. Install with: pip install mlflow")
except Exception as e:
    print(f"⚠️ Error connecting to MLflow: {e}")

## 9. Analytics: DuckDB for Fast Queries

### Why DuckDB?

**Performance**:
- **10-100x Faster**: Than pandas for analytical queries
- **SQL Interface**: Familiar SQL syntax for complex queries
- **Columnar Processing**: Optimized for analytical workloads
- **Zero Configuration**: In-process database, no server setup

**Use Cases**:
- Query training results across models
- Aggregate metrics by fold, model type, hyperparameters
- Join metadata with results
- Fast filtering and grouping

**Location**: `lib/utils/duckdb_analytics.py`

In [ ]:
# DuckDB Analytics Example
try:
    from lib.utils.duckdb_analytics import DuckDBAnalytics
    
    analytics = DuckDBAnalytics()
    
    # Register metadata tables
    if scaled_metadata.exists():
        analytics.register_parquet('videos', str(scaled_metadata))
        print("✅ Registered 'videos' table")
    
    if features_metadata.exists():
        analytics.register_parquet('features', str(features_metadata))
        print("✅ Registered 'features' table")
    
    # Example query: Class distribution
    if scaled_metadata.exists():
        result = analytics.query("""
            SELECT 
                label,
                COUNT(*) as count,
                ROUND(100.0 * COUNT(*) / SUM(COUNT(*)) OVER (), 2) as percentage
            FROM videos
            GROUP BY label
            ORDER BY count DESC
        """)
        
        if result is not None and len(result) > 0:
            print("\n📊 Class Distribution (DuckDB Query):")
            display(result)
    
    print("\n💡 Example DuckDB Queries:")
    print("```python")
    print("# Query training results")
    print("analytics.register_parquet('results', 'data/stage5/*/metrics.json')")
    print("result = analytics.query('""")
    print("    SELECT model_type, AVG(test_f1) as avg_f1, STDDEV(test_f1) as std_f1")
    print("    FROM results")
    print("    GROUP BY model_type")
    print("    ORDER BY avg_f1 DESC")
    print("""")")
    print("```")
    
except ImportError:
    print("⚠️ DuckDB not installed. Install with: pip install duckdb")
except Exception as e:
    print(f"⚠️ Error using DuckDB: {e}")

## 10. Orchestration: Apache Airflow DAGs

### Pipeline Orchestration

**Apache Airflow DAG**: `airflow/dags/fvc_pipeline_dag.py`

**Pipeline Stages as Tasks**:
1. **Stage 1 Task**: Video augmentation (parallelizable)
2. **Stage 2 Task**: Feature extraction (depends on Stage 1)
3. **Stage 3 Task**: Video scaling (depends on Stage 1, parallel with Stage 2)
4. **Stage 4 Task**: Scaled feature extraction (depends on Stage 3)
5. **Stage 5 Task**: Model training (depends on Stages 2, 3, 4)

**Dependency Graph**:
```
Stage 1
  ├─> Stage 2
  └─> Stage 3 ─> Stage 4
  └─> Stage 5 (depends on 2, 3, 4)
```

**Benefits**:
- **Dependency Management**: Automatic task ordering
- **Retry Logic**: Automatic retries on failure (1 retry, 5min delay)
- **Monitoring**: Web UI for pipeline status
- **Scheduling**: Cron-based scheduling support
- **Parallelization**: Parallel stage execution where possible
- **Checkpointing**: Resume from failures

**Location**: `airflow/dags/fvc_pipeline_dag.py`

In [ ]:
# Airflow DAG visualization
airflow_dag_path = project_root / "airflow" / "dags" / "fvc_pipeline_dag.py"

if airflow_dag_path.exists():
    print("✅ Airflow DAG found:")
    print(f"   Location: {airflow_dag_path}")
    
    # Read and display DAG structure
    with open(airflow_dag_path, 'r') as f:
        dag_code = f.read()
    
    print("\n📋 DAG Tasks:")
    print("   1. stage1_augmentation")
    print("   2. stage2_features (depends on stage1)")
    print("   3. stage3_scaling (depends on stage1)")
    print("   4. stage4_scaled_features (depends on stage3)")
    print("   5. stage5_training (depends on stage2, stage3, stage4)")
    
    print("\n💡 To use Airflow:")
    print("```bash")
    print("# Start Airflow webserver")
    print("airflow webserver --port 8080")
    print("")
    print("# Start Airflow scheduler")
    print("airflow scheduler")
    print("")
    print("# Trigger DAG")
    print("airflow dags trigger fvc_pipeline")
    print("```")
else:
    print("⚠️ Airflow DAG not found")
    print(f"   Expected: {airflow_dag_path}")

## 11. Model Evaluation & Results

### Evaluation Metrics

**Classification Metrics** (per fold, then averaged):
- **Accuracy**: Overall correctness
- **F1 Score**: Harmonic mean of precision and recall (primary metric)
- **Precision**: True positives / (True positives + False positives)
- **Recall**: True positives / (True positives + False negatives)
- **AUC-ROC**: Area under ROC curve
- **Confusion Matrix**: Per-class error analysis

**Cross-Validation Statistics**:
- **Mean**: Average across 5 folds
- **Std**: Standard deviation (measures consistency)
- **Min/Max**: Best and worst fold performance

### Results Visualization

**Location**: `src/dashboard_results.py` (Streamlit dashboard)

In [ ]:
# Load and visualize results
if stage5_dir.exists():
    import json
    
    results_summary = []
    
    for model_dir in sorted(stage5_dir.iterdir()):
        if not model_dir.is_dir():
            continue
        
        model_name = model_dir.name
        
        # Find metrics file
        metrics_files = list(model_dir.glob("**/metrics.json"))
        
        if metrics_files:
            metrics_file = metrics_files[0]
            
            try:
                with open(metrics_file, 'r') as f:
                    metrics = json.load(f)
                
                # Extract summary metrics
                if 'mean_test_f1' in metrics:
                    results_summary.append({
                        'Model': model_name,
                        'Mean F1': metrics.get('mean_test_f1', 0),
                        'Std F1': metrics.get('std_test_f1', 0),
                        'Mean Accuracy': metrics.get('mean_test_acc', 0),
                        'Mean Precision': metrics.get('mean_test_precision', 0),
                        'Mean Recall': metrics.get('mean_test_recall', 0)
                    })
            except Exception as e:
                print(f"⚠️ Could not load metrics for {model_name}: {e}")
    
    if results_summary:
        results_df = pd.DataFrame(results_summary)
        results_df = results_df.sort_values('Mean F1', ascending=False)
        
        print("📊 Model Performance Summary:")
        display(results_df.style.format({
            'Mean F1': '{:.4f}',
            'Std F1': '{:.4f}',
            'Mean Accuracy': '{:.4f}',
            'Mean Precision': '{:.4f}',
            'Mean Recall': '{:.4f}'
        }).background_gradient(subset=['Mean F1'], cmap='RdYlGn').set_table_styles([
            {'selector': 'th', 'props': [('background-color', '#2196F3'), ('color', 'white'), ('font-weight', 'bold')]}
        ]))
        
        # Visualization
        fig, axes = plt.subplots(1, 2, figsize=(14, 5))
        
        # F1 Score comparison
        axes[0].barh(results_df['Model'], results_df['Mean F1'], 
                    xerr=results_df['Std F1'], capsize=5, color='#4CAF50')
        axes[0].set_xlabel('F1 Score', fontsize=12)
        axes[0].set_title('Model Performance (F1 Score)', fontsize=14, fontweight='bold')
        axes[0].grid(axis='x', alpha=0.3)
        
        # Accuracy comparison
        axes[1].barh(results_df['Model'], results_df['Mean Accuracy'], color='#2196F3')
        axes[1].set_xlabel('Accuracy', fontsize=12)
        axes[1].set_title('Model Performance (Accuracy)', fontsize=14, fontweight='bold')
        axes[1].grid(axis='x', alpha=0.3)
        
        plt.tight_layout()
        plt.show()
    else:
        print("⚠️ No metrics found in trained models")

## 12. Production Deployment Considerations

### Model Serving

**Options**:
- **MLflow Model Serving**: Built-in serving for PyTorch models
- **TorchServe**: PyTorch's production serving framework
- **FastAPI**: Custom REST API with model loading
- **ONNX Export**: Cross-platform deployment

### Monitoring

**MLflow Model Registry**:
- Version control for models
- Staging → Production promotion
- A/B testing support

**Custom Monitoring**:
- Prediction logging
- Performance metrics tracking
- Drift detection

### Scalability

**Batch Processing**:
- Process videos in batches
- Use GPU clusters for inference
- Parallelize across multiple GPUs

**Real-Time Processing**:
- Frame-by-frame processing
- Streaming inference
- Low-latency requirements

---

## Conclusion

This pipeline demonstrates a **production-grade ML system** with:
- ✅ **Comprehensive Data Processing**: 5-stage pipeline from raw videos to trained models
- ✅ **23 Diverse Models**: From baselines to state-of-the-art architectures
- ✅ **MLOps Infrastructure**: MLflow, Airflow, DuckDB integration
- ✅ **Best Practices**: 5-fold CV, hyperparameter optimization, experiment tracking
- ✅ **Production-Ready**: Error handling, checkpointing, reproducibility

**Next Steps**:
1. Review individual model notebooks (5a-5u) for detailed architecture
2. Explore MLflow UI for experiment comparison
3. Use DuckDB for custom analytics queries
4. Deploy best model to production using MLflow Model Registry